# خرچ کے دعوے کا تجزیہ

یہ نوٹ بک دکھاتی ہے کہ کس طرح ایجنٹس کو پلگ انز کے ذریعے مقامی رسیدوں کی تصاویر سے سفر کے اخراجات کو پروسیس کرنے، خرچ کے دعوے کے ای میل تیار کرنے، اور پائی چارٹ کے ذریعے خرچ کے ڈیٹا کو بصری طور پر پیش کرنے کے لیے بنایا جا سکتا ہے۔ ایجنٹس کام کے سیاق و سباق کے مطابق خود بخود فنکشنز کا انتخاب کرتے ہیں۔

مراحل:
1. OCR ایجنٹ مقامی رسید کی تصویر کو پروسیس کرتا ہے اور سفر کے اخراجات کا ڈیٹا نکالتا ہے۔
2. ای میل ایجنٹ خرچ کے دعوے کا ای میل تیار کرتا ہے۔

### سفر کے اخراجات کے منظر کی مثال:
تصور کریں کہ آپ ایک ملازم ہیں جو کسی دوسرے شہر میں کاروباری ملاقات کے لیے سفر کر رہے ہیں۔ آپ کی کمپنی کی پالیسی ہے کہ تمام معقول سفر سے متعلق اخراجات کی واپسی کی جائے۔ یہاں ممکنہ سفر کے اخراجات کی تفصیل دی گئی ہے:

- **ٹرانسپورٹیشن:**
آپ کے گھر کے شہر سے منزل کے شہر تک آنے جانے کے لیے ہوائی سفر۔
ایئرپورٹ آنے اور جانے کے لیے ٹیکسی یا رائیڈ ہیلنگ سروسز۔
منزل کے شہر میں مقامی ٹرانسپورٹ (جیسے پبلک ٹرانزٹ، کرائے کی گاڑیاں، یا ٹیکسیز)۔

- **رہائش:**
میٹنگ کے مقام کے قریب درمیانے درجے کے کاروباری ہوٹل میں تین راتوں کا قیام۔

- **کھانے:**
کمپنی کی یومیہ الاؤنس پالیسی کے مطابق ناشتے، دوپہر کے کھانے، اور رات کے کھانے کے لیے یومیہ کھانے کا الاؤنس۔

- **متفرق اخراجات:**
ایئرپورٹ پر پارکنگ فیس۔
ہوٹل میں انٹرنیٹ تک رسائی کے چارجز۔
ٹپس یا چھوٹے سروس چارجز۔

- **دستاویزات:**
آپ تمام رسیدیں (فلائٹس، ٹیکسیز، ہوٹل، کھانے وغیرہ) اور مکمل شدہ خرچ کی رپورٹ واپسی کے لیے جمع کراتے ہیں۔


## ضروری لائبریریاں درآمد کریں

نوٹ بک کے لیے ضروری لائبریریاں اور ماڈیولز درآمد کریں۔


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## اخراجات کے ماڈلز کی وضاحت کریں

انفرادی اخراجات کے لیے ایک Pydantic ماڈل بنائیں اور ایک ExpenseFormatter کلاس تیار کریں جو صارف کی درخواست کو منظم اخراجاتی ڈیٹا میں تبدیل کرے۔

ہر خرچ کو اس فارمیٹ میں پیش کیا جائے گا:  
`{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## ایجنٹس کی تعریف - ای میل تیار کرنا

ایک ایجنٹ کلاس بنائیں جو اخراجات کے دعوے جمع کروانے کے لیے ایک ای میل تیار کرے۔
- یہ ایجنٹ `kernel_function` ڈیکوریٹر کا استعمال کرتا ہے تاکہ ایک فنکشن کو متعین کرے جو اخراجات کے دعوے جمع کروانے کے لیے ایک ای میل تیار کرے۔
- یہ اخراجات کی کل رقم کا حساب لگاتا ہے اور تفصیلات کو ای میل کے متن میں ترتیب دیتا ہے۔


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# رسید کی تصاویر سے سفر کے اخراجات نکالنے کے لیے ایجنٹ

ایک ایجنٹ کلاس بنائیں جو رسید کی تصاویر سے سفر کے اخراجات نکال سکے۔
- یہ ایجنٹ `kernel_function` ڈیکوریٹر استعمال کرتا ہے تاکہ ایک فنکشن کو تعریف کرے جو رسید کی تصاویر سے سفر کے اخراجات نکالے۔
- رسید کی تصویر کو OCR (آپٹیکل کریکٹر ریکگنیشن) کے ذریعے متن میں تبدیل کریں اور متعلقہ معلومات جیسے تاریخ، تفصیل، رقم، اور زمرہ نکالیں۔


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## اخراجات کی پروسیسنگ

ایک غیر ہم وقت ساز فنکشن کی وضاحت کریں تاکہ اخراجات کو پروسیس کیا جا سکے، ضروری ایجنٹس کو تخلیق اور رجسٹر کر کے، اور پھر انہیں فعال کر کے۔

- یہ فنکشن اخراجات کو پروسیس کرتا ہے، ماحولیاتی متغیرات کو لوڈ کر کے، ضروری ایجنٹس کو تخلیق کر کے، اور انہیں پلگ ان کے طور پر رجسٹر کر کے۔
- یہ دو ایجنٹس کے ساتھ ایک گروپ چیٹ تخلیق کرتا ہے اور ایک پرامپٹ پیغام بھیجتا ہے تاکہ اخراجات کے ڈیٹا کی بنیاد پر ای میل اور پائی چارٹ تیار کیا جا سکے۔
- یہ چیٹ کے دوران پیش آنے والی کسی بھی خرابی کو ہینڈل کرتا ہے اور ایجنٹس کی مناسب صفائی کو یقینی بناتا ہے۔


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## مرکزی فنکشن

مرکزی فنکشن کو اس طرح ترتیب دیں کہ یہ کنسول کو صاف کرے اور `process_expenses` فنکشن کو غیر ہم وقتی طور پر چلائے۔


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**ڈس کلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے پوری کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہیں۔ اصل دستاویز، جو اس کی اصل زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
